In [ ]:
import hail as hl
hl.init(default_reference='GRCh38')

Loading BokehJS ...

/Users/pine/miniforge3/envs/spark/lib/python3.9/site-packages/hail/context.py:350: UserWarning:

Using hl.init with a default_reference argument is deprecated. To set a default reference genome after initializing hail, call `hl.default_reference` with an argument to set the default reference genome.

Running on Apache Spark version 3.5.5
SparkUI available at http://10.203.123.237:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.134-952ae203dbbe
LOGGING: writing to /Users/pine/Desktop/trgn516/ances_stuff/hail-20250408-1252-0.2.134-952ae203dbbe.log


2025-04-08 12:52:33.815 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'
2025-04-08 12:52:37.297 Hail: INFO: scanning VCF for sortedness...
2025-04-08 12:52:39.322 Hail: INFO: Coerced prefix-sorted VCF, requiring additional sorting within data partitions on each query.
2025-04-08 12:52:46.622 Hail: INFO: Coerced sorted dataset
2025-04-08 12:53:33.170 Hail: INFO: Coerced sorted dataset
2025-04-08 12:57:58.437 Hail: INFO: Coerced sorted dataset
2025-04-08 13:03:07.601 Hail: INFO: Coerced sorted dataset
2025-04-08 13:03:29.698 Hail: INFO: Coerced sorted dataset
2025-04-08 13:11:14.688 Hail: INFO: Coerced sorted dataset
2025-04-08 13:11:24.493 Hail: INFO: Coerced sorted dataset
2025-04-08 13:11:34.481 Hail: INFO: Coerced sorted dataset
2025-04-08 13:11:44.824 Hail: INFO: Coerced sorted dataset
2025-04-08 13:34:08.816 Hail: INFO: Coerced sorted dataset


In [24]:
pca_loadings = hl.read_table("/Users/pine/Desktop/trgn516/ances_stuff/pca_loading/gnomad.v4.0.pca_loadings.ht")
sarek_mt = hl.import_vcf("/Users/pine/Desktop/trgn516/ances_stuff/normal_1.strelka.variants.vcf", reference_genome='GRCh38', array_elements_required=False)
pca_loadings.describe()
sarek_mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'loadings': array<float64> 
    'pca_af': float64 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------
----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        BLOCKAVG_min30p3a: bool, 
        CIGAR: array<str>, 
        END: int32, 
        IDREP: array<int32>, 
        MQ: float64, 
        REFREP: array<int32>, 
        RU: array<str>, 
        SNVHPOL: int32
    }
----------------------------------------
Entry fields:
    'AD': array<int32>
    'ADF': array<int32>
    'ADR': array<int3

In [ ]:
### DO NOT run this cell
# Different ref version, need to Liftover from hg19 to GRCh38
# download from gs
# gsutil cp gs://hail-common/references/grch37_to_grch38.over.chain.gz .

#rg37 = hl.get_reference('GRCh37')  
#rg38 = hl.get_reference('GRCh38')  

# need to comment this line after one use if you need to rerun cell after encountering an error
#rg37.add_liftover('/Users/pine/Desktop/trgn516/ances_stuff/grch37_to_grch38.over.chain.gz', rg38)  

#sarek_mt_hg38 = sarek_mt.annotate_rows(new_locus=hl.liftover(sarek_mt.locus, 'GRCh38'))  
#sarek_mt_hg38 = sarek_mt_hg38.filter_rows(hl.is_defined(sarek_mt_hg38.new_locus))
#sarek_mt_hg38 = sarek_mt_hg38.key_rows_by(locus=sarek_mt_hg38.new_locus, alleles=sarek_mt_hg38.alleles) 


In [ ]:
# if you wish to see detail information run this cell
#sarek_mt.summarize()
#pca_loadings.summarize()

In [3]:
import onnx
from gnomad.sample_qc.ancestry import (
    apply_onnx_classification_model,
    assign_population_pcs,
)
from gnomad.utils.filtering import filter_to_adj
from gnomad_qc.v4.resources.basics import get_checkpoint_path

onnx_rf = "/Users/pine/Desktop/trgn516/ances_stuff/gnomad.v4.0.RF_fit.onnx"

with hl.hadoop_open(onnx_rf, "rb") as f:
    onx_fit = onnx.load(f)


pcs = hl.experimental.pc_project(sarek_mt.GT, pca_loadings.loadings, pca_loadings.pca_af)

In [ ]:
ht, model = assign_population_pcs(
    pcs,
    pc_cols=pcs.scores[:20], # define number of pc, depend on the loaded model
    fit=onx_fit,
    # Using a low min_prob (like 0.01) is permissive: almost everyone gets a label. 
    # High values (e.g. 0.9) are conservative: only very confident calls are made.
    min_prob=0.01, 
    apply_model_func=apply_onnx_classification_model,
)


ht.aggregate(hl.agg.counter(ht.pop))

INFO (gnomad.sample_qc.ancestry 371): Found the following sample count after population assignment: nfe: 1


{'nfe': 1}

In [14]:
ht.describe()
ht.summarize()
ht.show()

----------------------------------------
Global fields:
    'assign_pops_from_pc_params': struct {
        min_assignment_prob: float64
    } 
----------------------------------------
Row fields:
    's': str 
    'pca_scores': array<float64> 
    'pop': str 
    'prob_afr': float64 
    'prob_ami': float64 
    'prob_amr': float64 
    'prob_asj': float64 
    'prob_eas': float64 
    'prob_fin': float64 
    'prob_mid': float64 
    'prob_nfe': float64 
    'prob_sas': float64 
----------------------------------------
Key: ['s']
----------------------------------------


Non-missing,1 (100.00%)
Missing,0
Min Size,17
Max Size,17
Mean Size,17.00
Sample Values,['patient1_normal_1']
Non-missing,1 (100.00%)
Missing,0
Min Size,20
Max Size,20
Mean Size,20.00


,,,,,,,,,,,
s,pca_scores,pop,prob_afr,prob_ami,prob_amr,prob_asj,prob_eas,prob_fin,prob_mid,prob_nfe,prob_sas
str,array<float64>,str,float64,float64,float64,float64,float64,float64,float64,float64,float64
"""patient1_normal_1""","[7.91e-03,2.08e-02,-8.13e-03,-3.65e-03,-9.32e-03,1.54e-03,-1.84e-02,1.92e-03,1.24e-03,2.45e-03,-6.42e-03,3.71e-04,-5.47e-04,6.31e-04,-1.34e-03,1.96e-03,-3.17e-03,1.16e-02,3.12e-03,-8.68e-03]","""nfe""",4.00e-02,1.00e-02,6.00e-02,1.20e-01,3.00e-02,2.00e-02,1.00e-02,6.90e-01,2.00e-02


In [ ]:
from bokeh.io import show

# Extract the first two PCs and population labels
# Plot results for population assignment in pca
p = hl.plot.scatter(ht.pca_scores[0], ht.pca_scores[1],
                    label=ht.pop,
                    title='PCA', xlabel='PC1', ylabel='PC2',
                    n_divisions=None)
show(p)
